I saved the lat and lon from North York, this is going to be the point of interest from now on.

In [7]:
from geopy.geocoders import Nominatim

address = 'North York, CA'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North York are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of North York are 43.7543263, -79.44911696639593.


I generate a map, centered in North Yor and with all the neighbourhoods from there.

In [8]:
import folium # map rendering library

northyork_data = df_merged.loc[df.Borough == 'North York', :].reset_index(drop = True)

# create map of Manhattan using latitude and longitude values
map_northyork = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(northyork_data['Latitude'], northyork_data['Longitude'], northyork_data['Neighborhood']):
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = 'blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7,
        parse_html = False).add_to(map_northyork)  
    
map_northyork

I saved acces tokens to get acces (I remove them when I run everything) 

In [9]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
ACCESS_TOKEN = '' # your FourSquare Access Token
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

I used the function to get a new dataframe with all the venues in the proximity and related with the neighbourhoods.

In [10]:
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

def getNearbyVenues(names, latitudes, longitudes, radius = 700):
    
    venues_list = []
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'],
            v['venue']['id'],
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                             'Neighborhood Latitude', 
                             'Neighborhood Longitude',
                             'Venue',
                             'Id',
                             'Venue Latitude',
                             'Venue Longitude',
                             'Venue Category']
    
    return(nearby_venues)

# create a new dataframe with venues
northyork_venues = getNearbyVenues(names = northyork_data['Neighborhood'],
                                   latitudes = northyork_data['Latitude'],
                                   longitudes = northyork_data['Longitude']
                                  )

Parkwoods
Victoria Village
Lawrence Manor, Lawrence Heights
Don Mills
Glencairn
Don Mills
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Fairview, Henry Farm, Oriole
Northwood Park, York University
Bayview Village
Downsview
York Mills, Silver Hills
Downsview
North Park, Maple Leaf Park, Upwood Park
Humber Summit
Willowdale, Newtonbrook
Downsview
Bedford Park, Lawrence Manor East
Humberlea, Emery
Willowdale, Willowdale East
Downsview
York Mills West
Willowdale, Willowdale West


In [11]:
northyork_venues.shape

(371, 8)

In [12]:
northyork_venues

Neighborhood  Neighborhood Latitude  \
0                      Parkwoods              43.753259   
1                      Parkwoods              43.753259   
2                      Parkwoods              43.753259   
3                      Parkwoods              43.753259   
4               Victoria Village              43.725882   
..                           ...                    ...   
366  Willowdale, Willowdale West              43.782736   
367  Willowdale, Willowdale West              43.782736   
368  Willowdale, Willowdale West              43.782736   
369  Willowdale, Willowdale West              43.782736   
370  Willowdale, Willowdale West              43.782736   

     Neighborhood Longitude                   Venue                        Id  \
0                -79.329656         Brookbanks Park  4e8d9dcdd5fbbbb6b3003c7b   
1                -79.329656                PetSmart  4e13a489b0fb5dfdd0756c90   
2                -79.329656           Variety Store  4cb11e2075ebb60cd1c4caad   
3                -79.329656          Ranchdale Park  4fad6522e4b08da80a56db50   
4                -79.315572  Victoria Village Arena  4c633acb86b6be9a61268e34   
..                      ...                     ...                       ...   
366              -79.442259      Shoppers Drug Mart  4bf818505efe2d7ff5186a34   
367              -79.442259               Dollarama  5aea20d13731ee002c1cbe4a   
368              -79.442259             Tim Hortons  4bc75114af07a59340f07e2d   
369              -79.442259               Hartman's  4b58e4cdf964a520667128e3   
370              -79.442259            Antibes Park  4e9b748ee5e8dffcaeeebb99   

     Venue Latitude  Venue Longitude     Venue Category  
0         43.751976       -79.332140               Park  
1         43.748639       -79.333488          Pet Store  
2         43.751974       -79.333114  Food & Drink Shop  
3         43.751388       -79.322138               Park  
4         43.723481       -79.315635       Hockey Arena  
..              ...              ...                ...  
366       43.784847       -79.446028           Pharmacy  
367       43.784670       -79.446670     Discount Store  
368       43.780940       -79.444231        Coffee Shop  
369       43.784312       -79.446213            Butcher  
370       43.778872       -79.448705               Park  

[371 rows x 8 columns]

In [13]:
print('There are {} uniques categories.'.format(len(northyork_venues['Venue Category'].unique())))
print(northyork_venues.shape)

There are 122 uniques categories.
(371, 8)


I have printed a map with all the venues from North York.

In [15]:
import folium # map rendering library

# create map of Manhattan using latitude and longitude values
map_northyork = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, venue_name, neighborhood in zip(northyork_venues['Venue Latitude'], northyork_venues['Venue Longitude'], northyork_venues['Venue'], northyork_venues['Neighborhood']):
    label = 'Venue: {}; Neighborhood/s: {}'.format(venue_name, neighborhood)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [float(lat), float(lng)],
        radius = 5,
        popup = label,
        color = 'blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7,
        parse_html = False).add_to(map_northyork)  
    
map_northyork

I converted the venues dataframe in a dataframe with dummy variables and finally getting the percentage of venue category per neighbourhood.

In [16]:
# one hot encoding
northyork_onehot = pd.get_dummies(northyork_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
northyork_onehot['Neighborhood'] = northyork_venues['Neighborhood']

# move neighborhood column to the first column
fixed_columns = [northyork_onehot.columns[-1]] + list(northyork_onehot.columns[:-1])
northyork_onehot = northyork_onehot[fixed_columns]

northyork_onehot.head()

Neighborhood  Accessories Store  Airport  American Restaurant  \
0         Parkwoods                  0        0                    0   
1         Parkwoods                  0        0                    0   
2         Parkwoods                  0        0                    0   
3         Parkwoods                  0        0                    0   
4  Victoria Village                  0        0                    0   

   Art Gallery  Arts & Crafts Store  Asian Restaurant  Athletics & Sports  \
0            0                    0                 0                   0   
1            0                    0                 0                   0   
2            0                    0                 0                   0   
3            0                    0                 0                   0   
4            0                    0                 0                   0   

   Bagel Shop  Bakery  ...  Supplement Shop  Sushi Restaurant  Tea Room  \
0           0       0  ...                0                 0         0   
1           0       0  ...                0                 0         0   
2           0       0  ...                0                 0         0   
3           0       0  ...                0                 0         0   
4           0       0  ...                0                 0         0   

   Thai Restaurant  Theater  Toy / Game Store  Trail  Video Game Store  \
0                0        0                 0      0                 0   
1                0        0                 0      0                 0   
2                0        0                 0      0                 0   
3                0        0                 0      0                 0   
4                0        0                 0      0                 0   

   Vietnamese Restaurant  Women's Store  
0                      0              0  
1                      0              0  
2                      0              0  
3                      0              0  
4                      0              0  

[5 rows x 123 columns]

In [17]:
northyork_grouped = northyork_onehot.groupby('Neighborhood').mean().reset_index()
northyork_grouped.head()

Neighborhood  Accessories Store  \
0  Bathurst Manor, Wilson Heights, Downsview North                0.0   
1                                  Bayview Village                0.0   
2                Bedford Park, Lawrence Manor East                0.0   
3                                        Don Mills                0.0   
4                                        Downsview                0.0   

    Airport  American Restaurant  Art Gallery  Arts & Crafts Store  \
0  0.000000             0.000000     0.000000                  0.0   
1  0.000000             0.000000     0.000000                  0.0   
2  0.000000             0.033333     0.000000                  0.0   
3  0.000000             0.000000     0.032258                  0.0   
4  0.030303             0.000000     0.000000                  0.0   

   Asian Restaurant  Athletics & Sports  Bagel Shop    Bakery  ...  \
0          0.000000            0.000000    0.000000  0.000000  ...   
1          0.000000            0.000000    0.000000  0.000000  ...   
2          0.000000            0.000000    0.033333  0.033333  ...   
3          0.032258            0.032258    0.000000  0.000000  ...   
4          0.000000            0.030303    0.000000  0.030303  ...   

   Supplement Shop  Sushi Restaurant  Tea Room  Thai Restaurant  Theater  \
0              0.0          0.045455       0.0         0.000000      0.0   
1              0.0          0.000000       0.0         0.000000      0.0   
2              0.0          0.033333       0.0         0.033333      0.0   
3              0.0          0.000000       0.0         0.000000      0.0   
4              0.0          0.000000       0.0         0.000000      0.0   

   Toy / Game Store  Trail  Video Game Store  Vietnamese Restaurant  \
0          0.000000    0.0               0.0               0.000000   
1          0.000000    0.0               0.0               0.000000   
2          0.033333    0.0               0.0               0.000000   
3          0.000000    0.0               0.0               0.000000   
4          0.000000    0.0               0.0               0.090909   

   Women's Store  
0            0.0  
1            0.0  
2            0.0  
3            0.0  
4            0.0  

[5 rows x 123 columns]

In [18]:
num_top_venues = 10

for hood in northyork_grouped['Neighborhood']:
    temp = northyork_grouped[northyork_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})

In [19]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [20]:
import numpy as np

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = northyork_grouped['Neighborhood']

for ind in np.arange(northyork_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(northyork_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighborhood 1st Most Common Venue  \
0  Bathurst Manor, Wilson Heights, Downsview North           Coffee Shop   
1                                  Bayview Village                  Bank   
2                Bedford Park, Lawrence Manor East    Italian Restaurant   
3                                        Don Mills   Japanese Restaurant   
4                                        Downsview         Grocery Store   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0                   Bank         Shopping Mall           Bridal Shop   
1     Chinese Restaurant          Intersection          Skating Rink   
2            Coffee Shop        Sandwich Place        Cosmetics Shop   
3                    Gym           Coffee Shop            Restaurant   
4  Vietnamese Restaurant           Gas Station           Pizza Place   

  5th Most Common Venue 6th Most Common Venue    7th Most Common Venue  \
0              Pharmacy         Deli / Bodega              Pizza Place   
1   Japanese Restaurant         Grocery Store                     Café   
2                   Pub            Restaurant  Comfort Food Restaurant   
3            Beer Store        Discount Store           Sandwich Place   
4           Coffee Shop        Sandwich Place                     Park   

  8th Most Common Venue 9th Most Common Venue     10th Most Common Venue  
0      Community Center     Mobile Phone Shop  Middle Eastern Restaurant  
1    Miscellaneous Shop     Mobile Phone Shop              Movie Theater  
2             Juice Bar          Cupcake Shop                Pizza Place  
3    Chinese Restaurant  Caribbean Restaurant                       Café  
4        Discount Store  Outdoor Supply Store              Moving Target

KNEAREST NEIGHTBOURS

Now, I assigned every neughbourhood to a 3 labels depending on the Knearest neightbours model, and then showed a map with these labels

In [21]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 3

northyork_grouped_clustering = northyork_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters = kclusters, random_state = 43).fit(northyork_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0],
      dtype=int32)

In [22]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

northyork_merged = northyork_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
northyork_merged = northyork_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on = 'Neighborhood')

northyork_merged.head() # check the last columns!

Postal Code     Borough                      Neighborhood   Latitude  \
0         M3A  North York                         Parkwoods  43.753259   
1         M4A  North York                  Victoria Village  43.725882   
2         M6A  North York  Lawrence Manor, Lawrence Heights  43.718518   
3         M3B  North York                         Don Mills  43.745906   
4         M6B  North York                         Glencairn  43.709577   

   Longitude  Cluster Labels 1st Most Common Venue   2nd Most Common Venue  \
0 -79.329656             0.0                  Park               Pet Store   
1 -79.315572             0.0            Playground            Hockey Arena   
2 -79.464763             1.0        Clothing Store  Furniture / Home Store   
3 -79.352188             1.0   Japanese Restaurant                     Gym   
4 -79.445073             1.0           Pizza Place           Grocery Store   

  3rd Most Common Venue  4th Most Common Venue 5th Most Common Venue  \
0     Food & Drink Shop      Accessories Store        Massage Studio   
1          Intersection            Coffee Shop                  Park   
2    Miscellaneous Shop  Vietnamese Restaurant           Coffee Shop   
3           Coffee Shop             Restaurant            Beer Store   
4            Playground         Sandwich Place    Italian Restaurant   

           6th Most Common Venue      7th Most Common Venue  \
0  Paper / Office Supplies Store       Outdoor Supply Store   
1          Portuguese Restaurant              Moving Target   
2           Fast Food Restaurant              Bowling Alley   
3                 Discount Store             Sandwich Place   
4            Japanese Restaurant  Latin American Restaurant   

       8th Most Common Venue 9th Most Common Venue  \
0                  Nightclub         Moving Target   
1  Middle Eastern Restaurant    Miscellaneous Shop   
2             Discount Store                  Park   
3         Chinese Restaurant  Caribbean Restaurant   
4                Gas Station           Fish Market   

          10th Most Common Venue  
0                  Movie Theater  
1              Mobile Phone Shop  
2  Paper / Office Supplies Store  
3                           Café  
4                           Park

In [23]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# drop nan
northyork_merged = northyork_merged.dropna()

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(northyork_merged['Latitude'], northyork_merged['Longitude'], northyork_merged['Neighborhood'], northyork_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [25]:
!jupyter trust Toronto_assignment.ipynb

Notebook already signed: Toronto_assignment.ipynb
